<a href="https://colab.research.google.com/github/Edward-TL/amazon_scraper/blob/master/Edici%C3%B3n_de_fechas_y_urls_de_Brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Iniciador

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


#Importación

In [2]:
#For the system
import os

#Manage of time
from datetime import datetime, timedelta
from pytz import timezone
import time

#Manage of files
import pandas as pd
import csv

#scrap
from bs4 import BeautifulSoup
from openpyxl.workbook import Workbook
import requests

#Posicionamiento en carpeta

In [3]:
%pwd

'/content'

In [4]:
%cd 'drive/My Drive/Colab Notebooks/4SS/4SS_db'
!ls

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db
 Playground  'Team example'   testing  'testing 2'


In [5]:
%cd 'testing/Masters'

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters


In [6]:
%cd 'br/parquet'

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters/br/parquet


In [7]:
!ls

br-master_db_amazon-devices.parquet  br-master_db_hi.parquet
br-master_db_appliances.parquet      br-master_db_home.parquet
br-master_db_automotive.parquet      br-master_db_hpc.parquet
br-master_db_baby-products.parquet   br-master_db_kitchen.parquet
br-master_db_beauty.parquet	     br-master_db_lawn-and-garden.parquet
br-master_db_books.parquet	     br-master_db_mobile-apps.parquet
br-master_db_computers.parquet	     br-master_db_music.parquet
br-master_db_digital-text.parquet    br-master_db_office.parquet
br-master_db_dvd.parquet	     br-master_db_pet-products.parquet
br-master_db_electronics.parquet     br-master_db_sports.parquet
br-master_db_fashion.parquet	     br-master_db_toys.parquet
br-master_db_furniture.parquet	     br-master_db_videogames.parquet
br-master_db_grocery.parquet


#Diseño y Prueba de función

In [18]:
def standarize_dates(date):
    check = False
    try:
        date = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S.%f')
        check = True
    except ValueError:
        try:
            date = datetime.strptime(str(date), '%Y-%m-%d %H:%M:%S')
            check = True
        except:
            try:
                date = datetime.strptime(str(date), '%Y-%m-%d %H_%M')
                check = True
            except:
                pass
    if check == True:
        date = date.strftime('%Y-%m-%d %H:%M')
    
    return date

In [36]:
def edit_price(price):
    if type(price) is float:
        print('oh yeah')
        price = price/100
        return price

In [38]:
db_to_edit.dtypes

time                 object
Rank                float32
Product Names        object
Image urls           object
Product links        object
Stars               float64
Reviews             float64
Authors/Company      object
Edition/Console      object
Price_std_or_min    float64
Max_prices           object
dtype: object

In [62]:
entry = 'br-master_db_music.parquet'
db_to_edit = pd.read_parquet(entry)

#Standarize dates
db_to_edit['time'] = db_to_edit['time'].apply(standarize_dates)
#Correct the product links
db_to_edit['Product links'] = db_to_edit['Product links'].str.replace('amazon.com.mx', 'amazon.com.br', regex=False)
#Sort values by date and Rank
db_to_edit.sort_values(by=['time', 'Rank'], inplace=True)
#Reset index
db_to_edit.reset_index(inplace=True, drop=True)
#Correction of prices
db_to_edit.loc[db_to_edit['time'] < '2020-08-17 15:03', 'Price_std_or_min'] = db_to_edit['Price_std_or_min']/100 

db_to_edit.to_excel('br-master_db_music_2.xlsx', index=False)
print('save')

save


In [63]:
!ls

br-master_db_amazon-devices.parquet  br-master_db_hi.parquet
br-master_db_appliances.parquet      br-master_db_home.parquet
br-master_db_automotive.parquet      br-master_db_hpc.parquet
br-master_db_baby-products.parquet   br-master_db_kitchen.parquet
br-master_db_beauty.parquet	     br-master_db_lawn-and-garden.parquet
br-master_db_books.parquet	     br-master_db_mobile-apps.parquet
br-master_db_computers.parquet	     br-master_db_music_2.xlsx
br-master_db_digital-text.parquet    br-master_db_music.parquet
br-master_db_dvd.parquet	     br-master_db_office.parquet
br-master_db_electronics.parquet     br-master_db_pet-products.parquet
br-master_db_fashion.parquet	     br-master_db_sports.parquet
br-master_db_furniture.parquet	     br-master_db_toys.parquet
br-master_db_grocery.parquet	     br-master_db_videogames.parquet


In [59]:
db_to_edit

,time,Rank,Product Names,Image urls,Product links,Stars,Reviews,Authors/Company,Edition/Console,Price_std_or_min,Max_prices
0,2020-07-31 00:39,1.0,"Ratos de Porão, LP ""Brasil""- Série Clássicos e...",https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Ratos-Por%C3%A3o-Bra...,NaN,NaN,Ratos de Porão,Disco de Vinil,104.00,None
1,2020-07-31 00:39,2.0,Alucinação - Série Clássicos em Vinil [Disco d...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/DISCO-DE-VINIL-BELCH...,NaN,NaN,Belchior,Disco de Vinil,110.00,None
2,2020-07-31 00:39,3.0,"Novos Baianos, Acabou Chorare- Série Clássicos...",https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Disco-Vinil-Baianos-...,NaN,NaN,Novos Baianos,Disco de Vinil,120.00,None
3,2020-07-31 00:39,4.0,Fifth Harmony - Reflection [CD],https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Reflection-Fifth-Har...,NaN,NaN,Fifth Harmony,CD de áudio,11.90,None
4,2020-07-31 00:39,5.0,The Dark Side Of The Moon [Disco de Vinil],https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Dark-Side-Moon-Disco...,NaN,NaN,Pink Floyd,Disco de Vinil,200.71,None
...,...,...,...,...,...,...,...,...,...,...,...
8295,2020-08-17 15:03,46.0,Amor Inventado [CD],https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Amor-Inventado-John-...,5.0,1.0,Karina Zeviani,CD de áudio,19.90,None
8296,2020-08-17 15:03,47.0,Kessous [CD],https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Kessous-Monique/dp/B...,NaN,NaN,Monique Kessous,CD de áudio,9.90,None
8297,2020-08-17 15:03,48.0,Lorena Chaves,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/SISTEMA-GLOBO-DE-GRA...,5.0,1.0,Lorena Chaves,CD de áudio,12.90,None
8298,2020-08-17 15:03,49.0,"Mutantes, LP Os Mutantes - Série Clássicos Em ...",https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Mutantes-LP-Os-S%C3%...,4.5,22.0,Mutantes,Disco de Vinil,132.00,None


#Funcion para edicion

In [112]:
def edit_df(df, country):
    #Standarize dates
    df['time'] = df['time'].apply(standarize_dates)

    if country is 'br':
        #Correct the product links
        try:
            df['Product links'] = df['Product links'].str.replace('amazon.com.mx', 'amazon.com.br', regex=False)
        except:
            df['Product links'].apply(urls_edition)
            
    #Sort values by date and Rank
    df.sort_values(by=['time', 'Rank'], inplace=True)
    #Reset index
    df.reset_index(inplace=True, drop=True)

    if country is 'br':
        #Correction of prices
        df.loc[df['time'] < '2020-08-17 15:03', 'Price_std_or_min'] = df['Price_std_or_min']/100 
        df.loc[df['Max_prices'] > 10, 'Max_prices'] = df['Max_prices']/100 
        df.loc[(df['Max_prices'] > 0) & (df['Max_prices'] < 10), 'Max_prices'] = df['Max_prices']*1000 
    
    return df

#Centered len

In [87]:
def centered_len(max_len):
    def centered_string(string):
        if type(string) != 'string':
            string = str(string)
            
        rest = max_len - len(string)
        n_spaces = int(rest/2)

        string_spaces = ' ' * n_spaces
        
        centered_string = string_spaces + string + string_spaces

        if len(centered_string)<max_len:
            centered_string = ' ' + centered_string

        return centered_string
    return centered_string

In [ ]:
db_to_edit = pd.read_parquet(entry)
db_to_edit = edit_df(db_to_edit, 'br')
db_to_edit

In [72]:
db_to_edit

#Editor y guardado

In [76]:
%cd '..'

/content/drive/My Drive/Colab Notebooks/4SS/4SS_db/testing/Masters


In [77]:
!ls

br  master_db_logs.csv	mx


In [114]:
country = 'br'

dir = f'{country}/parquet'
databases = os.listdir(dir)
n_databases = len(databases)

#initialize
df = [None]*n_databases
largest_len_db = 0
n = 0

for file_name in databases:
    db = f'{dir}/{file_name}'
    dataframe = pd.read_parquet(db)
    df[n] = db
    if len(file_name) > largest_len_db:
        largest_len_db = len(file_name)
    n = n + 1

#Print tools
centered_db = centered_len(largest_len_db)
centered_header = centered_db('masterdbs')
dashes = '-'*largest_len_db
print(f' {centered_header} | Edition')
print(f' {dashes} | ------ ')

for d in range(len(df)):
    #file name extract
    dir_len = len(dir)+1
    parquet_len = len('.parquet')
    file_name_len = len(df[d])-parquet_len
    file_name = df[d][dir_len:file_name_len]
    #File name print to log
    print(f' {centered_db(file_name)}', end=' | ')

    #--------Edition--------
    #Load Dataframe
    file_to_edit = pd.read_parquet(df[d])

    edit_df(file_to_edit, country)

    #--------Upadting--------
    #File names
    csv_file = f'{country}/csv/{file_name}.csv'
    excel_file = f'{country}/excel/{file_name}.xlsx'
    parquet_file = f'{country}/parquet/{file_name}.parquet'
    #File updating
    file_to_edit.to_csv(csv_file,sep="|")
    file_to_edit.to_excel(excel_file, index=False)
    file_to_edit.to_parquet(parquet_file, allow_truncated_timestamps=True)

    print('Done')

               masterdbs              | Edition
 ------------------------------------ | ------ 
          br-master_db_books          | Done
       br-master_db_digital-text      | Done
        br-master_db_videogames       | Done
      br-master_db_baby-products      | Done
        br-master_db_appliances       | Done
        br-master_db_computers        | Done
          br-master_db_beauty         | Done
        br-master_db_automotive       | Done
      br-master_db_amazon-devices     | Done
           br-master_db_toys          | Done
          br-master_db_sports         | Done
       br-master_db_pet-products      | Done
         br-master_db_grocery         | Done
          br-master_db_music          | Done
          br-master_db_office         | Done
           br-master_db_hpc           | Done
         br-master_db_fashion         | Done
       br-master_db_mobile-apps       | Done
         br-master_db_kitchen         | Done
     br-master_db_lawn-and-garden     | Done
    

In [103]:
def urls_edition(url):
    if type(url) is str:
        link = url.replace('amazon.com.mx', 'amazon.com.br')
        return link

In [110]:
entry = 'br/parquet/br-master_db_amazon-devices.parquet'
df = pd.read_parquet(entry)

df['time'] = df['time'].apply(standarize_dates)

if country is 'br':
    #Correct the product links
    try:
        df['Product links'] = df['Product links'].str.replace('amazon.com.mx', 'amazon.com.br', regex=False)
    except:
        df['Product links'].apply(urls_edition)

#Sort values by date and Rank
df.sort_values(by=['time', 'Rank'], inplace=True)
#Reset index
df.reset_index(inplace=True, drop=True)

if country is 'br':
    #Correction of prices
    df.loc[df['time'] < '2020-08-17 15:03', 'Price_std_or_min'] = df['Price_std_or_min']/100 
    df.loc[df['Max_prices'] > 0, 'Max_prices'] = df['Max_prices']/100 
    df.loc[(df['Max_prices'] > 0) & (df['Max_prices'] < 10), 'Max_prices'] = df['Max_prices']*1000 

df

,time,Rank,Product Names,Image urls,Product links,Stars,Reviews,Authors/Company,Edition/Console,Price_std_or_min,Max_prices
0,2020-07-31 00:38,1.0,Echo Dot (3ª Geração): Smart Speaker com Alexa...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Echo-Dot-3%C2%AA-Ger...,NaN,NaN,None,None,249.00,NaN
1,2020-07-31 00:38,2.0,Kindle 10a. geração com iluminação embutida –...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Staging-Product-Not-...,NaN,NaN,None,None,349.00,NaN
2,2020-07-31 00:38,3.0,Kindle Paperwhite 32 GB - Agora à prova d´água,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Novo-Kindle-Paperwhi...,NaN,NaN,None,None,499.00,649.0
3,2020-07-31 00:38,4.0,"Echo Show 5 - Smart Speaker com tela de 5,5"" e...",https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Echo-Show-Smart-Spea...,NaN,NaN,None,None,499.00,NaN
4,2020-07-31 00:38,5.0,Capa Nupro para Kindle Paperwhite - Cor Azul,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Capa-Nupro-para-Kind...,NaN,NaN,None,None,79.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8395,2020-08-17 15:03,46.0,Capa de Couro para Novo Oasis - Cor Preta,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Capa-Couro-para-Novo...,4.0,37.0,None,None,NaN,NaN
8396,2020-08-17 15:03,47.0,"Kindle Oasis Wi-Fi, iluminação embutida, tela ...",https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Kindle-ilumina%C3%A7...,3.9,142.0,None,None,NaN,NaN
8397,2020-08-17 15:03,48.0,Suporte Capa Para Celular Bicicleta Moto Prova...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Suporte-Prova-D%C3%A...,NaN,NaN,None,None,48.99,NaN
8398,2020-08-17 15:03,49.0,Suporte Apoio Stand De Parede Amazon Echo Dot ...,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.com.br/Suporte-Apoio-Stand-...,NaN,NaN,None,None,22.00,NaN
